# Tickable 8 - model solution

*&#169; Eike Mueller, University of Bath 2019-2021. This model solution is copyright of Eike Mueller, University of Bath. It is provided exclusively for educational purposes at the University and is to be downloaded or copied for your private study only. Further distribution, e.g. by upload to external repositories, is prohibited.*

### &#9745; Task 1
This is easily shown by inserting the proposed solution into the equations:
#### System A:
$x=2, y=0$:
$$
\begin{aligned}
x + y  &= 2 + 0 = 2\\
x + (1+\delta)y &= 2 + (1+\delta)\cdot 0 = 2\\
\end{aligned}
$$
#### System B:
$x=1, y=1$:
$$
\begin{aligned}
x + y  &= 1 + 1 = 2\\
x + (1+\delta)y &= 1 + (1+\delta)\cdot 1 = 2+\delta\\
\end{aligned}
$$

### &#9745; Task 2
The Python function `solve_systems()` can be implemented like this:

In [ ]:
import numpy as np
def solve_systems(delta):
    M = np.array([[1,1],[1,1+delta]])
    a = np.array([2,2])
    b = np.array([2,2+delta])
    solution_A = np.linalg.solve(M,a)
    solution_B = np.linalg.solve(M,b)
    return solution_A, solution_B

For $\delta=0.01$ we get, as expected:

In [ ]:
solution_A, solution_B = solve_systems(0.01)
print ('solution of system A : ',solution_A)
print ('solution of system B : ',solution_B)

### &#9745; Task 3
To fill the table, we can write a for loop to print out the solutions as follows:

#### Task 3a

In [ ]:
for delta in (0.01,1.E-6,1.E-7,1.E-8,1.E-12,1.E-14,1.E-15):
    solution_A, solution_B = solve_systems(delta)
    print ('delta = ',delta,' : ',solution_A,solution_B)

We get the following table:

| value of $$\delta$$ | Solution of System A | Solution of System B |
| --- | --- | --- |
| $$0.1$$ | $$2.\qquad 0.$$ |  $$1.\qquad 1.$$ |
| $$10^{-6}$$ | $$2.\qquad 0.$$ |  $$1.\qquad 1.$$ |
| $$10^{-7}$$ | $$2.\qquad 0.$$ |  $$1.\qquad 1.$$ |
| $$10^{-8}$$ | $$2.\qquad 0.$$ |  $$1.\qquad 1.$$ |
| $$10^{-14}$$ | $$2.\qquad 0.$$ |  $$0.97777778 \qquad 1.02222222$$ |
| $$10^{-15}$$ | $$2.\qquad 0.$$ |  $$1.2\qquad 0.8$$ |

While the solution of System A is correct for all values of $\delta$, System B is not solved correctly for small $\delta$. For $\delta=10^{-14}$ we get a solution is still close to the exact solution, but for $\delta = 10^{-15}$ the result is completely wrong.

#### Task 3b
The following function is almost identical to `solve_systems()`, but the matrix $M$ and the two vectors $\boldsymbol{a}$ and $\boldsymbol{b}$ are stored as 32bit numbers by adding the `dtype=np.float32` parameter to `np.array()`:

In [3]:
import numpy as np
def solve_systems_sp(delta):
    M = np.array([[1,1],[1,1+delta]],dtype=np.float32)
    a = np.array([2,2],dtype=np.float32)
    b = np.array([2,2+delta],dtype=np.float32)
    solution_A = np.linalg.solve(M,a)
    solution_B = np.linalg.solve(M,b)
    return solution_A, solution_B

Again we can fill the table by calling this function in a for-loop:

In [ ]:
for delta in (0.01,1.E-6,1.E-7,1.E-8,1.E-12,1.E-14,1.E-15):
    solution_A, solution_B = solve_systems_sp(delta)
    print ('delta = ',delta,' : ',solution_A,solution_B)

However, now the behaviour is different: While the solution of both System A and System B is correct for $\delta=0.01$ and $\delta=10^{-6}$, so $\delta=10^{-7}$ we get $x=2, y=0$ for both systems, and this clearly can't be correct. Furthermore, for smaller values of $\delta$ Python aborts with an error message:

```
LinAlgError: Singular matrix
```

A singular matrix is a matrix with determinant 0 which can not be inverted.

Although in exact arithmetic the matrix has a non-zero determinant for all values of $\delta$, this is not true in floating point arithmetic. Because of rounding errors in the computation of the matrix determinant, Python believes that the matrix has a zero determinant and refuses to invert it.

### &#9745; Task 4
We get the following results why trying to compute $F_{100}$ with `fib_iter` and `fib_rec_mat`:

In [1]:
from model_solutions import fib_iter, fib_rec_mat
print('F_{1000} [fib_iter]    = ',fib_iter(100))
print('F_{1000} [fib_rec_mat] = ',fib_rec_mat(100))

F_{1000} [fib_iter]    =  354224848179261915075
F_{1000} [fib_rec_mat] =  3736710778780434371


The result obtained with `fib_rec_mat` is incorrect. This is because `fib_rec_mat_vec` uses repeated multiplication with a numpy matrix to construct the vector $\boldsymbol{f}_n=\begin{pmatrix}F_n\\F_{n-1}\end{pmatrix}$. Since the entries of the matrix are of type `np.int64`, this will generate overflow errors of they are larger than $2^{63}-1$. To address this error, we can can check whether one of the entries of the vector $\boldsymbol{f}_n$ is larger than $2^{62}-1$, and trigger an exception if this is the case. Here is a modified implementation which takes this into account:

In [ ]:
import numpy as np
def fib_rec_mat_vec_with_overflow_handling(n):
    '''Compute the n-th Fibonacci vector (F_n, F_{n-1})

    input: Number n
    output: n-th Fibonacci vector (F_n, F_{n-1})
    '''
    if not (n == int(n)):
        raise Exception('Fibonacci number F_n not defined for non-integer values')
    elif (n < 0):
        raise Exception('Fibonacci number F_n not defined for negative values')
    elif (n == 0):
        return np.array([0,1])
    else:
        A_fib = np.array([[1,1],[1,0]])
        f_n = fib_rec_mat_vec_with_overflow_handling(n-1)
        if (f_n[0]>2**62-1) or (f_n[1]>2**62-1):
            raise Exception('Warning: potential overflow error')
        return A_fib@f_n
    
def fib_rec_mat_with_overflow_handling(n):
    '''Compute the n-th Fibonacci number by extracting the first component
    of the n-th Fibonacci vector (F_n, F_{n-1})

    input: Number n
    output: n-th Fibonacci number F_n
    '''
    if not (n == int(n)):
        raise Exception('Fibonacci number F_n not defined for non-integer values')
    elif (n < 0):
        raise Exception('Fibonacci number F_n not defined for negative values')
    f_fib = fib_rec_mat_vec_with_overflow_handling(n)
    return f_fib[0]

Instead of quietly returning an incorrect result, this implementation now aborts with a warning:

In [ ]:
fib_rec_mat_with_overflow_handling(100)

## Other things to try out
The determinant of the matrix $M$ is $\det(M) = 1\cdot (1+\delta) - 1 \cdot 1 = (1+\delta) - 1$. In exact arithmetic this reduces to $\delta$, but if $\delta$ is smaller than the machine epsilon, $1+\delta$ will be rounded to $1$, so the determinant becomes $0$ and Python thinks that the matrix is singular. This explains the error message in Task 3b: For single precision (32bit) floating point numbers $\epsilon_{\text{mach}}^{(\text{sp})}=2^{-24}\approx5.58\cdot 10^{-8}>\delta$ for $\delta\le 10^{-8}$. We do not encounter this problem in Task 3a, since for double precision (64bit) floating point numbers $\epsilon_{\text{mach}}^{(\text{sp})}=2^{-53}\approx1.11\cdot 10^{-16}<\delta$ for all $\delta\ge 10^{-15}$.

In floating point arithmetic, $1+\delta$ will be represented by some number $1+\delta'$, where usually $\delta'\ne\delta$, but $\delta'\approx \delta$. The *relative* difference between $\delta'$ and $\delta$ increases for small values of $\delta$. Bearing this in mind, the inverse of the matrix $M$ is (in floating point arithmetic):

$$
M^{-1} = \frac{1}{\delta'}
\begin{pmatrix}
1+\delta' & -1 \\
-1 & 1
\end{pmatrix}
$$

This gives the following solutions for the two systems:
#### System A
$$
\begin{pmatrix}
x\\y
\end{pmatrix}
=
\frac{1}{\delta'}
\begin{pmatrix}
1+\delta' & -1 \\
-1 & 1
\end{pmatrix}
\begin{pmatrix}
2\\2
\end{pmatrix}
=
\frac{1}{\delta'}
\begin{pmatrix}
(1+\delta')\cdot 2 - 1\cdot 2 \\
- 2
\end{pmatrix}
=
\frac{1}{\delta'}
\begin{pmatrix}
2\cdot(1+\delta') - 1\cdot 2 \\
-2 + 2
\end{pmatrix}
=
\begin{pmatrix}
2 \\
0
\end{pmatrix}
$$

We always get the correct solution $x=2,y=0$, irrespective of the value of  $\delta$.

#### System B
When solving this system, we need to bear in mind that the value of $2+\delta$ on the right hand side of the second equation gets rounded to $2+\delta''$ where $\delta''\approx \delta$, but (in general) $\delta''\ne \delta$ and $\delta''\ne \delta'$. Again, the *relative* error between $\delta''$ and $\delta$ increases for small values of $\delta$. In floating point arithmetic this results in the following solution of the System B:

$$
\begin{pmatrix}
x\\y
\end{pmatrix}
=
\frac{1}{\delta'}
\begin{pmatrix}
1+\delta' & -1 \\
-1 & 1
\end{pmatrix}
\begin{pmatrix}
2\\2+\delta''
\end{pmatrix}
=
\frac{1}{\delta'}
\begin{pmatrix}
(1+\delta')\cdot 2 - 1\cdot (2+\delta'') \\
-2 + (2+\delta'')
\end{pmatrix}
=
\frac{1}{\delta'}
\begin{pmatrix}
2\delta' - \delta'' \\
\delta''
\end{pmatrix}
=
\begin{pmatrix}
(2\delta' - \delta'')/\delta' \\
\delta''/\delta'
\end{pmatrix}
\ne
\begin{pmatrix}
1 \\
1
\end{pmatrix}
$$

The discrepancy between the numerical solution $x_{\text{num}}=(2\delta' - \delta'')/\delta'$, $y_{\text{num}}\delta''/\delta'$ and the true solution $x=1,y=1$ increases for small values of $\delta$, but it is small for large values of $\delta$ where $\delta\approx \delta'\approx \delta''$.